# AI Agents Crash Course - Part 2: Implementation Notebook

This notebook demonstrates the advanced concepts from Part 2:
- Modular crew architecture with YAML configuration
- Structured output generation with Pydantic
- Custom tool development
- Production-ready patterns

**Prerequisites:**
1. Install requirements: `pip install -r requirements.txt`
2. Set up `.env` file with your API keys
3. Ensure all project files are in the correct structure

## 1. Setup and Environment

In [32]:
# Install dependencies (uncomment if needed)
# !pip install crewai crewai-tools pydantic python-dotenv requests PyYAML

In [ ]:
import os
from dotenv import load_dotenv
from crewai import LLM

# Load environment variables
load_dotenv()

# Option D: Azure OpenAI
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

llm = LLM(
    model="azure/gpt-4o-mini",
    api_key=openai_api_key,
    base_url=openai_endpoint,
    api_version=openai_api_version,
    azure=True
)

print("🚀 Environment configured!")
print(f"LLM Model: {llm.model}")

## 2. YAML Configuration Overview

Let's examine the YAML configuration files that separate our agent definitions from code logic.

In [ ]:
import yaml

# Load and display agent configuration
with open('crews/config/agents.yaml', 'r') as file:
    agents_config = yaml.safe_load(file)

print("📋 Available Agents:")
for agent_name, config in agents_config.items():
    print(f"\n🤖 {agent_name}:")
    print(f"   Role: {config['role']}")
    print(f"   Goal: {config['goal'][:100]}..." if len(config['goal']) > 100 else f"   Goal: {config['goal']}")

In [ ]:
# Load and display task configuration
with open('crews/config/tasks.yaml', 'r') as file:
    tasks_config = yaml.safe_load(file)

print("📋 Available Tasks:")
for task_name, config in tasks_config.items():
    print(f"\n📝 {task_name}:")
    print(f"   Agent: {config['agent']}")
    description = config['description'][:150] + "..." if len(config['description']) > 150 else config['description']
    print(f"   Description: {description}")
    if 'depends_on' in config:
        print(f"   Depends on: {config['depends_on']}")

## 3. Basic Research Crew Implementation

Let's start with the basic research crew from the article.

In [ ]:
# Import our custom research crew
from crews.research_crew import ResearchCrew

# Create research crew instance
research_crew = ResearchCrew(llm = llm)
print("✅ Research crew initialized!")

In [ ]:
# Run a research task
topic = "The impact of AI on job markets"

print(f"🔍 Researching topic: {topic}")
print("This may take a few minutes...\n")

# Create and run the crew
result = research_crew.crew().kickoff(inputs={"topic": topic})

In [ ]:
from IPython.display import Markdown, display


# Display the research result
display(Markdown(result.raw))

## 4. Structured Output with Pydantic

Now let's demonstrate structured output generation using Pydantic schemas.

In [28]:
from crewai import Agent

from pydantic import BaseModel, Field

class EntityRelationEntity(BaseModel):
    entity: str = Field(description="The first entity in the triplet")
    relation: str = Field(description="The relation between the first and second entity")
    entity: str = Field(description="The second entity in the triplet")


In [26]:
from crewai import Agent

agent = Agent(
    role="Senior Linguist",
    goal="Analyse the query and extract entity-relation-entity triplets",
    backstory="You are a senior linguist that is known for your analytical skills.",
    verbose=True,
    llm=llm
)

In [ ]:
from crewai import Task

task = Task(
    description="""Analyse the query and return structured JSON
                   output in the form of
                   - entity
                   - relation
                   - entity

                   The query is: {query}
                   """,
    expected_output="""A structured JSON object with the
                       entity-relation-entity triplets""",
    output_pydantic=EntityRelationEntity,
    verbose=True,
    agent=agent
)

In [ ]:
from crewai import Crew, Process

crew = Crew(
    agents=[agent],
    tasks=[task],
    process=Process.sequential,
    verbose=True
)

response = crew.kickoff(inputs={"query": "Paris is the capital of France."})


In [ ]:
print(response.raw)

## 5. Custom Tools Development

Let's explore the custom currency conversion tool.

In [ ]:
# Import and test custom tools
from custom_tools import CurrencyConverterTool, CalculatorTool

# Test currency converter tool directly
if os.getenv('EXCHANGE_RATE_API_KEY'):
    print("💱 Testing Currency Converter Tool...")
    currency_tool = CurrencyConverterTool()
    
    # Test conversion
    result = currency_tool._run(amount=100, from_currency="USD", to_currency="EUR")
    print(f"✅ Result: {result}")
else:
    print("⚠️ Exchange Rate API key not found. Skipping currency tool test.")

# Test calculator tool
print("\n🧮 Testing Calculator Tool...")
calc_tool = CalculatorTool()
calc_result = calc_tool._run("2 + 2 * 3")
print(f"✅ Calculation result: {calc_result}")

In [ ]:
# Currency conversion with AI agent
if os.getenv('EXCHANGE_RATE_API_KEY'):
    print("🤖 Running Currency Conversion with AI Agent...")
    
    # Create currency conversion crew
    crew = research_crew.create_currency_crew(
        amount=100, 
        from_currency="USD", 
        to_currency="EUR"
    )
    
    result = crew.kickoff(inputs={
        "amount": 100,
        "from_currency": "USD",
        "to_currency": "EUR"
    })
    
    print("\n💰 Currency Analysis Result:")
    display(Markdown(result.raw))
else:
    print("⚠️ Exchange Rate API key required for currency conversion demo")

## 6. Advanced Multi-Agent Pipeline

Let's create a complete pipeline that processes natural language queries and performs currency conversion.

In [ ]:
# Advanced pipeline: Natural Language Query → Structured Parsing → Currency Conversion
from crewai import Agent, Task, Crew, Process
from custom_tools import CurrencyConverterTool
from structured_output_demo import CurrencyQuery

def create_advanced_currency_pipeline(user_query: str):
    """Create a pipeline that handles natural language currency queries."""
    
    # Step 1: Query Parser Agent
    query_parser = Agent(
        role="Query Parser",
        goal="Extract structured information from natural language queries",
        backstory="You are a linguistic expert who specializes in parsing user queries to extract relevant parameters.",
        verbose=True
    )
    
    # Step 2: Currency Analyst Agent (only if we have the API key)
    if os.getenv('EXCHANGE_RATE_API_KEY'):
        currency_analyst = Agent(
            role="Currency Analyst",
            goal="Provide real-time currency conversions and financial insights",
            backstory="You are a finance expert with deep knowledge of global exchange rates.",
            tools=[CurrencyConverterTool()],
            verbose=True
        )
    
    # Task 1: Parse the query
    parsing_task = Task(
        description=f"""Parse this natural language query about currency conversion:
        "{user_query}"
        
        Extract the amount, source currency, and target currency.
        If any information is missing, make reasonable assumptions.""",
        expected_output="A structured JSON object with currency conversion parameters",
        output_pydantic=CurrencyQuery,
        agent=query_parser
    )
    
    tasks = [parsing_task]
    agents = [query_parser]
    
    # Task 2: Perform conversion (only if API key available)
    if os.getenv('EXCHANGE_RATE_API_KEY'):
        conversion_task = Task(
            description="Use the parsed information to perform currency conversion and provide financial insights.",
            expected_output="A detailed response with conversion result and financial context",
            agent=currency_analyst
        )
        tasks.append(conversion_task)
        agents.append(currency_analyst)
    
    return Crew(
        agents=agents,
        tasks=tasks,
        process=Process.sequential,
        verbose=True
    )

print("✅ Advanced pipeline function created!")

In [ ]:
# Test the advanced pipeline
test_queries = [
    "How much is 100 dollars in euros today?",
    "Convert 50 pounds to yen",
    "What's 200 euros worth in Canadian dollars?"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*50}")
    print(f"Test {i}: {query}")
    print(f"{'='*50}")
    
    crew = create_advanced_currency_pipeline(query)
    result = crew.kickoff()
    
    print(f"\n📊 Final Result:")
    if hasattr(result, 'raw'):
        display(Markdown(result.raw))
    else:
        print(result)
    
    # Only test first query to avoid rate limits
    break

## 7. Configuration-Driven Development Demo

Let's demonstrate how easy it is to modify agent behavior by changing YAML configuration.

In [ ]:
# Create a modified agent configuration
modified_config = {
    'research_agent': {
        'role': 'Senior Market Research Analyst',
        'goal': 'Conduct deep market analysis and provide strategic insights on given topics',
        'backstory': 'You are a senior analyst with 15 years of experience in market research, specializing in technology trends and economic impacts.',
        'verbose': True
    }
}

print("📝 Original Research Agent:")
print(f"Role: {agents_config['research_agent']['role']}")
print(f"Goal: {agents_config['research_agent']['goal'][:100]}...")

print("\n📝 Modified Research Agent:")
print(f"Role: {modified_config['research_agent']['role']}")
print(f"Goal: {modified_config['research_agent']['goal'][:100]}...")

print("\n✅ This demonstrates how YAML configuration allows easy agent modification without code changes!")

## 8. Production Patterns and Best Practices

Let's review the production-ready patterns implemented in this project.

In [ ]:
# Check project structure
import os

def check_project_structure():
    """Verify project follows the recommended structure."""
    
    expected_files = [
        'requirements.txt',
        '.env.example', 
        'research_crew.py',
        'custom_tools.py',
        'structured_output_demo.py',
        'config/agents.yaml',
        'config/tasks.yaml'
    ]
    
    print("📁 Project Structure Check:")
    for file_path in expected_files:
        if os.path.exists(file_path):
            print(f"✅ {file_path}")
        else:
            print(f"❌ {file_path} (missing)")
    
    print("\n📋 Best Practices Implemented:")
    practices = [
        "✅ Configuration separated from logic (YAML files)",
        "✅ Environment variables for sensitive data",
        "✅ Modular crew architecture with CrewBase",
        "✅ Custom tools with proper error handling",
        "✅ Structured outputs with Pydantic schemas",
        "✅ Multiple implementation patterns (decorator & traditional)",
        "✅ Comprehensive examples and documentation"
    ]
    
    for practice in practices:
        print(practice)

check_project_structure()

## 9. Performance and Monitoring

Let's add some basic performance monitoring to our agents.

In [ ]:
import time
from datetime import datetime

def monitor_crew_performance(crew, inputs=None):
    """Monitor crew execution time and basic metrics."""
    
    print(f"🚀 Starting crew execution at {datetime.now().strftime('%H:%M:%S')}")
    start_time = time.time()
    
    # Execute crew
    if inputs:
        result = crew.kickoff(inputs=inputs)
    else:
        result = crew.kickoff()
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    print(f"\n📊 Performance Metrics:")
    print(f"⏱️  Execution Time: {execution_time:.2f} seconds")
    print(f"🤖 Number of Agents: {len(crew.agents)}")
    print(f"📝 Number of Tasks: {len(crew.tasks)}")
    print(f"📄 Output Length: {len(str(result.raw))} characters")
    
    return result, execution_time

print("✅ Performance monitoring function ready!")

In [ ]:
# Test performance monitoring with a simple task
from crewai import Agent, Task, Crew, Process

# Create a simple test crew
test_agent = Agent(
    role="Test Agent",
    goal="Provide a brief summary of a given topic",
    backstory="You are a helpful assistant that provides concise information.",
    verbose=True
)

test_task = Task(
    description="Provide a brief 2-sentence summary about artificial intelligence.",
    expected_output="A concise 2-sentence summary",
    agent=test_agent
)

test_crew = Crew(
    agents=[test_agent],
    tasks=[test_task],
    process=Process.sequential,
    verbose=True
)

# Monitor performance
result, exec_time = monitor_crew_performance(test_crew)

print(f"\n📝 Test Result:")
print(result.raw)

## 10. Summary and Next Steps

This notebook has demonstrated all the key concepts from AI Agents Crash Course Part 2.

In [ ]:
# Summary of what we've accomplished
summary = """
# 🎉 AI Agents Part 2 - Implementation Complete!

## ✅ Concepts Demonstrated:

### 1. **Modular Architecture**
- YAML configuration files for agents and tasks
- Separation of configuration from execution logic
- Easy modification without code changes

### 2. **Structured Output Generation**
- Pydantic schemas for consistent data formats
- Entity-relation-entity extraction
- Query parsing with structured results

### 3. **Custom Tool Development**
- Currency converter with real API integration
- Calculator tool with safety checks
- Proper error handling and validation

### 4. **Production Patterns**
- Environment variable management
- Modular crew architecture
- Performance monitoring
- Multiple implementation approaches

### 5. **Advanced Pipelines**
- Multi-agent sequential processing
- Natural language query handling
- Structured data flow between agents

## 🚀 Next Steps:

1. **Experiment with different configurations** in the YAML files
2. **Build custom tools** for your specific use cases
3. **Implement error handling** and retry mechanisms
4. **Add logging and monitoring** for production deployment
5. **Scale to more complex multi-agent workflows**

## 📚 Key Takeaways:

- **Configuration-driven development** enables rapid iteration
- **Structured outputs** are essential for production systems
- **Custom tools** bridge AI agents with real-world systems
- **Modular architecture** supports scalable agent systems

Ready to build production-ready AI agent systems! 🎯
"""

display(Markdown(summary))

In [ ]:
# Final verification of all components
print("🔍 Final System Check:")
print(f"✅ Environment variables loaded: {len([k for k in os.environ.keys() if k.endswith('_API_KEY')])} API keys")
print(f"✅ Agent configurations: {len(agents_config)} agents defined")
print(f"✅ Task configurations: {len(tasks_config)} tasks defined")
print(f"✅ Custom tools: CurrencyConverter, Calculator")
print(f"✅ Structured outputs: EntityTriplets, CurrencyQuery schemas")
print(f"✅ Crew implementations: Research, Currency, Parsing crews")

print("\n🎯 All systems operational! Ready for production deployment.")